# Notebook 0 : Domain clustering + LDA first try

In [179]:
import glob
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
import gensim
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/meftahzi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
domain = pd.read_csv("uk/domaincodes/domaincodes.csv")

In [50]:
print(domain)

                        domain                        type  comm_pol
0              dailymail.co.uk           Tabloid newspaper       NaN
1                       rai.it          Public broadcaster       NaN
2                  reuters.com  Quality newspaper/magazine       NaN
3                       abc.es  Quality newspaper/magazine       NaN
4                 20minutos.es           Tabloid newspaper       NaN
..                         ...                         ...       ...
537  resistancerepublicaine.eu         Hyperpartisan media       NaN
538         paris-normandie.fr  Quality newspaper/magazine       NaN
539              epochtimes.fr         Hyperpartisan media       NaN
540              epochtimes.de         Hyperpartisan media       NaN
541        fivethirtyeight.com    Digital-born news outlet       NaN

[542 rows x 3 columns]


In [4]:
portal = pd.read_csv("uk/domaincodes/newsportals.csv")

In [5]:
portal

,domain
0,aol.co.uk/news
1,aol.com/article/news
2,aol.de/nachrichten
3,freenet.de/nachrichten
4,gmx.net/magazine/panorama
5,notizie.libero.it
6,mail.com/int/news
7,msn.com/en-gb/news
8,msn.com/en-us/news
9,msn.com/de-de/nachrichten


In [88]:
news = pd.DataFrame()
DATA_FILES = ['uk/newspaper.json']
for filepath in DATA_FILES:
    current_news = pd.read_json(filepath, lines=True)
    current_news['country'] = filepath[:2]
    news = news.append(current_news)
    

news.reset_index(inplace=True, drop=True)
news.head()

,title,text,authors,imgs,movies,meta_lang,meta_keywords,htmlfile,url,error,country
0,The harlequin ladybird is a clever little devil,"Wenlock Edge, Shropshire Used to control crop ...","[Paul Evans, Peter Brown, Helen Roy]",[https://sb.scorecardresearch.com/p?c1=2&c2=60...,None,en,"[Insects, Invasive species, Rural affairs, Aut...",GER_2019-03-15_37_web_page.html,www.theguardian.com/environment/2016/nov/02/vo...,NaN,uk
1,LIVE: NZ terror attacks - Gun law reforms with...,Image:\n\nA man lays flowers for victims of th...,None,[https://e3.365dm.com/19/03/768x432/skynews-ne...,None,en,[],GER_2019-03-15_71_web_page.html,news.sky.com/story/live-significant-number-kil...,NaN,uk
2,Page not found,Cheer up. It's not the end of the world...\n\n...,None,"[data:image/jpeg;base64,/9j/4QAYRXhpZgAASUkqAA...",None,en,[],GER_2019-03-15_487_web_page.html,www.channel4.com/programmes/catchup/2019/03/14/e4,NaN,uk
3,Young Sheldon,None,None,[https://ic.c4assets.com/brands/brooklyn-nine-...,None,en,[],GER_2019-03-15_488_web_page.html,www.channel4.com/programmes/young-sheldon/on-d...,NaN,uk
4,Sign in,Uncheck if you're using a shared or public com...,None,None,None,en,[],GER_2019-03-15_489_web_page.html,4id.channel4.com/login?redirectUrl=https://www...,NaN,uk


# 1.Cleaning the dataframe : 

### 1.1. Only English articles and get rid of nan values :

In [90]:
news = news[news["meta_lang"]=="en"]
#news['meta_keywords'] = news['meta_keywords'].apply(lambda y: np.nan if len(y)==1 else y)
#news.dropna(subset = ["meta_keywords"], inplace=True)

In [91]:
news.head()

,title,text,authors,imgs,movies,meta_lang,meta_keywords,htmlfile,url,error,country
0,The harlequin ladybird is a clever little devil,"Wenlock Edge, Shropshire Used to control crop ...","[Paul Evans, Peter Brown, Helen Roy]",[https://sb.scorecardresearch.com/p?c1=2&c2=60...,None,en,"[Insects, Invasive species, Rural affairs, Aut...",GER_2019-03-15_37_web_page.html,www.theguardian.com/environment/2016/nov/02/vo...,NaN,uk
1,LIVE: NZ terror attacks - Gun law reforms with...,Image:\n\nA man lays flowers for victims of th...,None,[https://e3.365dm.com/19/03/768x432/skynews-ne...,None,en,[],GER_2019-03-15_71_web_page.html,news.sky.com/story/live-significant-number-kil...,NaN,uk
2,Page not found,Cheer up. It's not the end of the world...\n\n...,None,"[data:image/jpeg;base64,/9j/4QAYRXhpZgAASUkqAA...",None,en,[],GER_2019-03-15_487_web_page.html,www.channel4.com/programmes/catchup/2019/03/14/e4,NaN,uk
3,Young Sheldon,None,None,[https://ic.c4assets.com/brands/brooklyn-nine-...,None,en,[],GER_2019-03-15_488_web_page.html,www.channel4.com/programmes/young-sheldon/on-d...,NaN,uk
4,Sign in,Uncheck if you're using a shared or public com...,None,None,None,en,[],GER_2019-03-15_489_web_page.html,4id.channel4.com/login?redirectUrl=https://www...,NaN,uk


In [51]:
news[news['url'].str.contains('rai.it')]

,title,text,authors,imgs,movies,meta_lang,meta_keywords,htmlfile,url,error,country,domain,subdomain,suffix


### 1.2. Getting the domains :

#### Add domain, subdomain names :

In [92]:
import tldextract
news['domain'] = news['url'].apply (lambda row: tldextract.extract(row).domain)
news['subdomain'] = news['url'].apply (lambda row: tldextract.extract(row).subdomain)
news['suffix'] = news['url'].apply (lambda row: tldextract.extract(row).suffix)

In [93]:
print('The number of unique URLs and domains  per country:')
news.agg({"url": "nunique", "domain": "nunique","subdomain": "nunique"})

The number of unique URLs and domains  per country:


url          92641
domain         179
subdomain      280
dtype: int64

In [13]:
news

,title,text,authors,imgs,movies,meta_lang,meta_keywords,htmlfile,url,error,country,domain,subdomain,suffix
0,The harlequin ladybird is a clever little devil,"Wenlock Edge, Shropshire Used to control crop ...","[Paul Evans, Peter Brown, Helen Roy]",[https://sb.scorecardresearch.com/p?c1=2&c2=60...,None,en,"[Insects, Invasive species, Rural affairs, Aut...",GER_2019-03-15_37_web_page.html,www.theguardian.com/environment/2016/nov/02/vo...,NaN,uk,theguardian,www,com
1,LIVE: NZ terror attacks - Gun law reforms with...,Image:\n\nA man lays flowers for victims of th...,None,[https://e3.365dm.com/19/03/768x432/skynews-ne...,None,en,[],GER_2019-03-15_71_web_page.html,news.sky.com/story/live-significant-number-kil...,NaN,uk,sky,news,com
2,Page not found,Cheer up. It's not the end of the world...\n\n...,None,"[data:image/jpeg;base64,/9j/4QAYRXhpZgAASUkqAA...",None,en,[],GER_2019-03-15_487_web_page.html,www.channel4.com/programmes/catchup/2019/03/14/e4,NaN,uk,channel4,www,com
3,Young Sheldon,None,None,[https://ic.c4assets.com/brands/brooklyn-nine-...,None,en,[],GER_2019-03-15_488_web_page.html,www.channel4.com/programmes/young-sheldon/on-d...,NaN,uk,channel4,www,com
4,Sign in,Uncheck if you're using a shared or public com...,None,None,None,en,[],GER_2019-03-15_489_web_page.html,4id.channel4.com/login?redirectUrl=https://www...,NaN,uk,channel4,4id,com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120578,"Giveaway: 65-Inch TCL TV, PS4 Pro, Nerf Fortni...",E3 2019 is finally here and we want to celebra...,[Tania Gonzalez],[https://gamespot1.cbsistatic.com/uploads/scre...,None,en,[],GER_2019-06-15_3508_mobile_.html,newsletter.gamespot.com/re?l=D0I2r8kcsfIrnbxytoI7,NaN,uk,gamespot,newsletter,com
120579,Amazon Echo price drops AGAIN as owners get a ...,Amazon Echo now gets voice calling from Vodafo...,[David Snelling],[https://cdn.images.express.co.uk/img/covers/7...,None,en,[],GER_2019-06-15_4600_mobile_.html,www.express.co.uk/life-style/science-technolog...,NaN,uk,express,www,co.uk
120580,Amazon Echo is FREE is this new deal as 15 new...,Amazon Echo is now free in this latest deal\n\...,[David Snelling],[https://cdn.images.express.co.uk/img/page/pla...,None,en,[],GER_2019-06-15_4601_mobile_.html,www.express.co.uk/life-style/science-technolog...,NaN,uk,express,www,co.uk
120583,"GIF API - Better, Faster & Free",Data Driven\n\nTenor takes a data driven appro...,None,[http://fox13now.com/assets/img/about/about-us...,None,en,[Tenor Free GIF Search API],GER_2019-06-15_8094_mobile_.html,api.tenor.com/,NaN,uk,tenor,api,com


In [14]:
domains_by_count = pd.DataFrame({"domain":news.domain.value_counts().index, "occurrence":news.domain.value_counts().values, "fraction": news.domain.value_counts().values/news.domain.value_counts().values.sum()})

In [15]:
domains_by_count

,domain,occurrence,fraction
0,bbc,35326,0.381321
1,theguardian,8696,0.093868
2,dailymail,7766,0.083829
3,sky,6473,0.069872
4,thesun,5127,0.055343
...,...,...,...
174,news5cleveland,1,0.000011
175,arcamax,1,0.000011
176,nbcdfw,1,0.000011
177,windfinder,1,0.000011


In [16]:
##a=news[['domain','subdomain']].groupby('domain')['subdomain'].nunique()
a=news[['domain','subdomain']].groupby('domain')
result=a['subdomain'].unique()
result=pd.DataFrame({"domain":result.index, "subdomain":result.values})



In [17]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(result)

                   domain                                          subdomain
0                     abc              [www, , mylogin, mobile, push, iview]
1                   abc13                                                 []
2                   abc15                                            [www, ]
3                    abc7                                                 []
4             abc7chicago                                                 []
5                  abc7ny                                                 []
6           abcactionnews                                            [, www]
7                    abcn                                                 []
8               aljazeera                                      [interactive]
9                 android              [developer, messages, market, source]
10       androidauthority                                            [www, ]
11                aptoide  [en, cdn-en, bitdefender-antivirus.en, armorfl...

In [18]:
df_row =pd.merge(domains_by_count, result, on='domain')

In [19]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print (df_row)

                   domain  occurrence  fraction  \
0                     bbc       35326  0.381321   
1             theguardian        8696  0.093868   
2               dailymail        7766  0.083829   
3                     sky        6473  0.069872   
4                  thesun        5127  0.055343   
5               telegraph        3071  0.033149   
6                 express        2392  0.025820   
7                   metro        2197  0.023715   
8                     itv        2053  0.022161   
9             independent        1691  0.018253   
10               thetimes        1629  0.017584   
11               channel4        1454  0.015695   
12                  inews        1350  0.014572   
13               buzzfeed        1215  0.013115   
14   thetelegraphandargus         897  0.009683   
15                nytimes         768  0.008290   
16               standard         552  0.005958   
17             kentonline         501  0.005408   
18                     ft      

In [52]:
k = 19
k_top = domains_by_count.head(k)
print(f'For top {k}, the fraction is ', k_top['fraction'].sum())

For top 19, the fraction is  0.9029479388175862


# 2. Extracting working subdomains for top 20 UK domains

In [180]:
# https://docs.google.com/document/d/1ZumFo_s8Urrl0H9sAx245fDHfmyA_tyaWQlnoqE_nqI/edit?usp=sharing
searchfor = ["www.bbc.co.uk/news","ablink.editorial.theguardian.com","www.theguardian.com/news/","www.theguardian.com/world/","www.theguardian.com/commentisfree","www.theguardian.com/us-news/","www.theguardian.com/uk-news/","www.theguardian.com/politics/","www.theguardian.com/education/","www.theguardian.com/society/","www.theguardian.com/science/","www.theguardian.com/business/","www.theguardian.com/money/","www.theguardian.com/sport/","www.theguardian.com/australia-news/","www.theguardian.com/travel/","www.theguardian.com/technology/","www.theguardian.com/commentisfree/","theguardian.com/news/","theguardian.com/world/","theguardian.com/commentisfree/","theguardian.com/us-news/","theguardian.com/uk-news/","theguardian.com/politics/","theguardian.com/education/","theguardian.com/society/","theguardian.com/science/","theguardian.com/business/","theguardian.com/money/","theguardian.com/sport/","theguardian.com/australia-news/","theguardian.com/travel/","theguardian.com/technology/","theguardian.com/commentisfree/","dailymail.co.uk/news/","dailymail.co.uk/columnists/","dailymail.co.uk/debate/","dailymail.co.uk/femail/","dailymail.co.uk/health/","dailymail.co.uk/money/","dailymail.co.uk/sport/","dailymail.co.uk/travel/","dailymail.co.uk/tvshowbiz/","www.dailymail.co.uk/news/","www.dailymail.co.uk/columnists/","www.dailymail.co.uk/debate/","www.dailymail.co.uk/femail/","www.dailymail.co.uk/health/","www.dailymail.co.uk/money/","www.dailymail.co.uk/sport/","www.dailymail.co.uk/travel/","www.dailymail.co.uk/tvshowbiz/","news.sky.com/story/","www.thesun.co.uk/fabulous/","www.thesun.co.uk/money/","www.thesun.co.uk/news/","www.thesun.co.uk/sport/","www.thesun.co.uk/travel/","www.thesun.co.uk/tech/","thesun.co.uk/fabulous/","thesun.co.uk/money/","thesun.co.uk/news/","thesun.co.uk/sport/","thesun.co.uk/travel/", "thesun.co.uk/tech/","www.telegraph.co.uk/news","www.telegraph.co.uk/politics/","express.co.uk/","www.express.co.uk/","metro.co.uk/","itv.com/news/","www.itv.com/news/","www.independent.co.uk/news/","independent.co.uk/news/","www.thetimes.co.uk/article/","www.channel4.com/news/","inews.co.uk/news/","thetelegraphandargus.co.uk/news/","www.thetelegraphandargus.co.uk/news/","www.nytimes.com/","www.nytimes.com/","www.standard.co.uk/news","standard.co.uk/news","kentonline.co.uk/","www.kentonline.co.uk/","ft.com/content/","www.ft.com/content","www.eveningtimes.co.uk/news/","eveningtimes.co.uk/news/"]
filtered=news[news['url'].str.contains('|'.join(searchfor))]

filtered.reset_index(drop=True)

,title,text,authors,imgs,movies,meta_lang,meta_keywords,htmlfile,url,error,country,domain,subdomain,suffix
0,LIVE: NZ terror attacks - Gun law reforms with...,Image:\n\nA man lays flowers for victims of th...,None,[https://e3.365dm.com/19/03/768x432/skynews-ne...,None,en,[],GER_2019-03-15_71_web_page.html,news.sky.com/story/live-significant-number-kil...,NaN,uk,sky,news,com
1,Australian senator blames immigration for mosq...,An Australian senator has provoked outrage aft...,[Jane Wharton],[https://sb.scorecardresearch.com/p?c1=2&c2=60...,None,en,[],GER_2019-03-15_1075_web_page.html,metro.co.uk/2019/03/15/australian-senator-blam...,NaN,uk,metro,,co.uk
2,Australian senator blames immigration for mosq...,An Australian senator has provoked outrage aft...,[Jane Wharton],[https://sb.scorecardresearch.com/p?c1=2&c2=60...,None,en,[],GER_2019-03-15_1076_web_page.html,metro.co.uk/2019/03/15/australian-senator-blam...,NaN,uk,metro,,co.uk
3,Christchurch shootings: 49 dead in New Zealand...,Media playback is unsupported on your device M...,None,[http://ssc.api.bbc.com/?c1=2&c2=19293874&ns_s...,None,en,[],GER_2019-03-15_1563_web_page.html,www.bbc.co.uk/news/world-asia-47578798,NaN,uk,bbc,www,co.uk
4,David Steel suspended by Liberal Democrats ove...,Image copyright Getty Images Image caption Lor...,None,[https://ichef.bbci.co.uk/news/320/cpsprodpb/1...,None,en,[],GER_2019-03-15_1564_web_page.html,www.bbc.co.uk/news/uk-scotland-scotland-politi...,NaN,uk,bbc,www,co.uk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35286,Conservative leadership field slims to six as ...,Conservative leadership field slims to six as ...,"[Greg Heffer, Political Reporter]",[https://e3.365dm.com/19/06/768x432/skynews-ma...,None,en,[],GER_2019-06-15_1622_mobile_.html,news.sky.com/story/conservative-leadership-fie...,NaN,uk,sky,news,com
35287,'Glazed ring' donut jokes gets city boss from ...,Sign me up for news alerts\n\nA city boss has ...,None,"[http://fox13now.com/1x1.gif, http://fox13now....",None,en,"[Business News, Quirky, Kent, People & Employm...",GER_2019-06-15_1623_mobile_.html,www.kentonline.co.uk/kent/news/boss-suspended-...,NaN,uk,kentonline,www,co.uk
35288,Former Arsenal footballer Ian Wright taught hi...,Ian Wright has opened up about warning his chi...,[Rishma Dosani],[https://sb.scorecardresearch.com/p?c1=2&c2=60...,[https://www.youtube.com/embed/E7Cs9MJnwuI?ver...,en,[],GER_2019-06-15_2463_mobile_.html,metro.co.uk/2019/06/15/ian-wright-taught-child...,NaN,uk,metro,,co.uk
35289,Amazon Echo price drops AGAIN as owners get a ...,Amazon Echo now gets voice calling from Vodafo...,[David Snelling],[https://cdn.images.express.co.uk/img/covers/7...,None,en,[],GER_2019-06-15_4600_mobile_.html,www.express.co.uk/life-style/science-technolog...,NaN,uk,express,www,co.uk
